In [1]:
import os
import json
import torch

from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from diffusers import StableDiffusionImg2ImgPipeline, AutoPipelineForImage2Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

/home/r12942159/miniconda3/envs/AI/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_images(img_dir):
    img_paths = [os.path.join(img_dir, f"{str(i).zfill(6)}.jpg") for i in range(1, 101)]
    images = [Image.open(path) for path in img_paths]

    return images

In [9]:
def load_phi4_model(phi4_model_name, custom_cache_dir):
    processor = AutoProcessor.from_pretrained(phi4_model_name, trust_remote_code=True, cache_dir=custom_cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        phi4_model_name, 
        device_map="cuda", 
        torch_dtype="auto", 
        trust_remote_code=True,
        _attn_implementation='flash_attention_2',
        cache_dir=custom_cache_dir,
    ).cuda()

    return processor, model

In [10]:
def generate_text_prompt(image, max_new_tokens, processor, model, phi4_model_name):
    instruction = "<|user|><|image_1|>Describe this person in the exact artistic style of Peanuts comics (Snoopy-style). Ensure the description makes the character look like they belong in a Charles Schulz comic strip. The person’s features—such as hair color, expression, and outfit—should remain the same, but they must be transformed into the signature Peanuts cartoon style: simple, bold outlines, flat colors, round heads, dot eyes, and minimal shading. The background should be minimalistic, similar to classic Peanuts comic settings. <|end|><|assistant|>"
    generation_config = GenerationConfig.from_pretrained(phi4_model_name)
    inputs = processor(text=instruction, images=image, return_tensors='pt').to('cuda:0')

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            generation_config=generation_config,
        )
        generated_ids = generated_ids[:, inputs['input_ids'].shape[1]:]
        prompt = processor.batch_decode(generated_ids, 
                                        skip_special_tokens=True,
                                        clean_up_tokenization_spaces=False,)
    
    return prompt

In [11]:
def generate_text_prompts(images, max_new_tokens, length, processor, model):
    prompts = []
    for i in tqdm(range(length)):
        prompt = generate_text_prompt(
            images[i],
            max_new_tokens, 
            processor, 
            model, 
            "microsoft/Phi-4-multimodal-instruct"
        )
        prompts.append(prompt)
    
    return prompts

In [12]:
def save_prompts2json(prompts, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump({"prompts": prompts}, f, indent=4)
    
    print(f"Prompts successfully saved to {filename}")

In [13]:
def load_stable_diffusion(model_id, cache_dir):
    # pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    #     model_id, 
    #     torch_dtype=torch.float16,
    #     cache_dir=cache_dir,
    #     safety_checker = None,
    #     feature_extractor = None,
    # ).to("cuda")

    pipe = AutoPipelineForImage2Image.from_pretrained(
        "stable-diffusion-v1-5/stable-diffusion-v1-5", 
        torch_dtype=torch.float16, 
        variant="fp16", 
        use_safetensors=False,
        cache_dir="/home/r12942159/data_18TB",
        safety_checker = None,
        feature_extractor = None,
    )
    pipe.enable_model_cpu_offload()

    return pipe

In [14]:
def read_prompts(prompt_path):
    with open(prompt_path, "r", encoding="utf-8") as f:
        prompts = json.load(f)

    return prompts['prompts']

In [15]:
def resize_image(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    return transform(image).permute(1, 2, 0).mul(255).byte().numpy()

In [60]:
def generate_stylized_images(pipe, prompt, image, strength=0.75, guidance_scale=7.5, seed=41):
    image = Image.fromarray(resize_image(image))
    image = pipe(
        prompt=prompt, 
        image=image, 
        strength=strength, 
        guidance_scale=guidance_scale, 
        generator=torch.manual_seed(seed),
    ).images[0]
    
    return image

In [169]:
def save_images(stylized_resize_images, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for i, resized_image in enumerate(stylized_resize_images):
        output_filename = f"{i+1:06d}.jpg"
        output_path = os.path.join(output_folder, output_filename)
        resized_image.save(output_path)
        
        print(f"Processed {output_filename}")

#### Main()

In [ ]:
phi4_processor, phi4_model = load_phi4_model(phi4_model_name = "microsoft/Phi-4-multimodal-instruct", 
                                             custom_cache_dir = "/home/r12942159/data_18TB")

images = read_images(img_dir = '/home/r12942159/NTU_AI/Hw1/content_image/')

prompts = generate_text_prompts(
    images, 
    75, 
    len(images), 
    phi4_processor, 
    phi4_model,
)
save_prompts2json(prompts, 'Task2-2_prompts.json')

In [ ]:
pipe = load_stable_diffusion(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", 
    "/home/r12942159/data_18TB",
)

images = read_images(img_dir = '/home/r12942159/NTU_AI/Hw1/content_image/')
prompts = read_prompts("/home/r12942159/NTU_AI/Hw1/Task2-2_prompts.json")

Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  7.65it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
stylized_resize_images = [generate_stylized_images(pipe, prompts[i], images[i], strength=0.75, guidance_scale=7.5, seed=41) for i in range(len(images))]
save_images(stylized_resize_images, '/home/r12942159/NTU_AI/Hw1/hw1_Task2-2_output/')

Processed 000001.jpg
Processed 000002.jpg
Processed 000003.jpg
Processed 000004.jpg
Processed 000005.jpg
Processed 000006.jpg
Processed 000007.jpg
Processed 000008.jpg
Processed 000009.jpg
Processed 000010.jpg
Processed 000011.jpg
Processed 000012.jpg
Processed 000013.jpg
Processed 000014.jpg
Processed 000015.jpg
Processed 000016.jpg
Processed 000017.jpg
Processed 000018.jpg
Processed 000019.jpg
Processed 000020.jpg
Processed 000021.jpg
Processed 000022.jpg
Processed 000023.jpg
Processed 000024.jpg
Processed 000025.jpg
Processed 000026.jpg
Processed 000027.jpg
Processed 000028.jpg
Processed 000029.jpg
Processed 000030.jpg
Processed 000031.jpg
Processed 000032.jpg
Processed 000033.jpg
Processed 000034.jpg
Processed 000035.jpg
Processed 000036.jpg
Processed 000037.jpg
Processed 000038.jpg
Processed 000039.jpg
Processed 000040.jpg
Processed 000041.jpg
Processed 000042.jpg
Processed 000043.jpg
Processed 000044.jpg
Processed 000045.jpg
Processed 000046.jpg
Processed 000047.jpg
Processed 000

#### Generate profile photo with snoopy style

In [ ]:
phi4_processor, phi4_model = load_phi4_model(phi4_model_name = "microsoft/Phi-4-multimodal-instruct", 
                                             custom_cache_dir = "/home/r12942159/data_18TB")

img = Image.open('/home/r12942159/NTU_AI/Hw1/profile_photo.jpg')

instruction = "<|user|><|image_1|>Describe this person in the exact artistic style of Peanuts comics (Snoopy-style). Ensure the description makes the character look like they belong in a Charles Schulz comic strip. The person’s features—such as hair color, expression, and outfit—should remain the same, but they must be transformed into the signature Peanuts cartoon style: simple, bold outlines, flat colors, round heads, dot eyes, and minimal shading. The background should be minimalistic, similar to classic Peanuts comic settings. <|end|><|assistant|>"
generation_config = GenerationConfig.from_pretrained("microsoft/Phi-4-multimodal-instruct")
inputs = phi4_processor(text=instruction, images=img, return_tensors='pt').to('cuda:0')

with torch.no_grad():
    generated_ids = phi4_model.generate(
        **inputs,
        max_new_tokens=512,
        generation_config=generation_config,
    )
    generated_ids = generated_ids[:, inputs['input_ids'].shape[1]:]
    prompt = phi4_processor.batch_decode(generated_ids, 
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False,)

In [58]:
img = Image.open('/home/r12942159/NTU_AI/Hw1/Task2-2_profile_photo1.jpg')
prompt = 'In the Peanuts comic style, the person would have a round head with short, dark hair. The expression would be a simple, neutral look, with a slight hint of a smile. The outfit would be a white shirt with blue horizontal stripes, but the colors would be more solid and less detailed. The background would be minimalistic, with a plain white backdrop to keep the focus on the character. The overall style would be simple, bold, and cartoonish, fitting the Peanuts aesthetic.'
generate_stylized_images(pipe, prompt, img, 0.85, 10.5, 41).save('/home/r12942159/NTU_AI/Hw1/Task2-2_profile_snoopy.jpg')  

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['focus on the character . the overall style would be simple , bold , and cartoonish , fitting the peanuts aesthetic .']
100%|██████████| 42/42 [00:00<00:00, 79.83it/s]
